In [1]:
import torch
import torch.nn as nn  
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.metrics import classification_report
import ResNet as resnet
import DenseNet as densenet
import EfficientNet as efficientnet
import MobileNetV2 as mobilenet
import ViT as vit
import solver.solver as solver
import solver.solver_v2 as solver_v2
import solver.solver_v3 as solver_v3
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator

/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

n_epochs = 30
batch_size = 128
learning_rate = 1e-4
diff_drop = "v3"
opacus_baseline = False

model = resnet.resnet18(num_classes=10, diff_drop=diff_drop)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [3]:
train_path = "/media/data1/hyunjun/cifar-10/train/"
test_path = "/media/data1/hyunjun/cifar-10/test/"

In [4]:
trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.491, 0.482, 0.447), (0.247, 0.243, 0.262)),
])

trainset = torchvision.datasets.ImageFolder(root=train_path, transform=trans)
testset = torchvision.datasets.ImageFolder(root=test_path, transform=trans)

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=False, )
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True, drop_last=False, )

classes = trainset.classes
print(classes)

privacy_engine = PrivacyEngine()
if opacus_baseline:
    if not ModuleValidator.is_valid(model):
        model = ModuleValidator.fix(model)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    model, optimizer, train_loader = privacy_engine.make_private(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        noise_multiplier=1.1,
        max_grad_norm=10.0,
    )
        
model = model.to(device)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/opacus/privacy_engine.py:143: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


In [5]:
def train_loop(dataloader, model, criterion, optimizer, epoch):
    size = len(dataloader.dataset)
    total_correct = 0
    total_loss = 0
    
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        model.train()
        correct = 0
        
        pred = None
        p = 0.0
        if diff_drop == "v3":
            pred, p = model(X.to(device), epoch)
        else:
            pred, p = model(X.to(device))
        
        loss = criterion(pred, y.to(device))
        correct += (torch.argmax(pred, dim=1) == y.to(device)).type(torch.float).sum().item()
        total_correct += correct
        
        model.train()
        total_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        
        if diff_drop == "v1":
            solver.PseudoPruning(model.fc, p)
        elif diff_drop == "v2":
            solver_v2.PseudoPruning(model.fc, p)
        elif diff_drop == "v3":
            solver_v3.PseudoPruning(model.fc, p)
        
        optimizer.step()
        
        if (batch % 50 == 0):
            print(f"Batch {batch:>5d}: Loss per batch is {loss.item():>7.6f} and Accuracy is {correct / batch_size:>7.6f}")
    print(f"For Iteration: Loss is {(total_loss / (size // batch_size)):>7.6f} and Accuracy is {total_correct / size:>7.6f}")

In [6]:
def test_loop(dataloader, model, criterion):
    test_loss = 0
    correct = 0

    model.eval()
    
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for X, y in dataloader:
            pred, _ = model(X.to(device))
            test_loss += criterion(pred, y.to(device)).item()
            
            pred = torch.argmax(pred, dim=1)
            correct += (pred == y.to(device)).type(torch.float).sum().item()
            
            pred = pred.cpu().numpy()
            y = y.cpu().numpy()
            y_true.extend(y)
            y_pred.extend(pred)
    
    print(classification_report(y_true=y_true, y_pred=y_pred, target_names=classes))
    
    return correct / len(dataloader.dataset)

In [7]:
max = 0.0
for epoch in range(n_epochs):
    print(f"Training: Epoch {epoch + 1} ------------------")
    train_loop(train_loader, model, criterion, optimizer, epoch + 1)
    print(f"----Test: Epoch {epoch + 1} ------------------")
    temp = test_loop(test_loader, model, criterion)
    
    # if temp > max:
    #     max = temp
    #     torch.save(model, './cifar10_effib0_v3.pt')

Training: Epoch 1 ------------------
Batch     0: Loss per batch is 2.349221 and Accuracy is 0.085938
Batch    50: Loss per batch is 1.778945 and Accuracy is 0.382812
Batch   100: Loss per batch is 1.571870 and Accuracy is 0.421875
Batch   150: Loss per batch is 1.430069 and Accuracy is 0.484375
Batch   200: Loss per batch is 1.429482 and Accuracy is 0.453125
Batch   250: Loss per batch is 1.451599 and Accuracy is 0.507812
Batch   300: Loss per batch is 1.305006 and Accuracy is 0.531250
Batch   350: Loss per batch is 1.279095 and Accuracy is 0.578125
For Iteration: Loss is 1.497885 and Accuracy is 0.454780
----Test: Epoch 1 ------------------
              precision    recall  f1-score   support

    airplane       0.58      0.55      0.56      1000
  automobile       0.45      0.85      0.59      1000
        bird       0.50      0.20      0.28      1000
         cat       0.41      0.12      0.18      1000
        deer       0.35      0.51      0.42      1000
         dog       0.53 

In [8]:
#torch.save(model, './cifar10_resnet18_v3_batch512.pt')